In [1]:
import pandas as pd
import re
from datetime import datetime,date
import random
import json
import os

In [2]:
df=pd.read_csv('../remain_quote.csv').drop(['id'],axis=1)

In [3]:
df=df.assign(
    words=lambda x:x.quote.str.split(' ')
).assign(
    len_quote=lambda x:x.words.str.len()
)

In [4]:
df_tmp=df.iloc[:50000][['quote','author','tags']]

In [5]:
def get_word_only(word):
    return re.findall('[a-zA-Z-]*',word)[0]

In [6]:
df_filterd=df_tmp.loc[lambda x:x.quote.str.len()<100].loc[lambda x:x.quote.str.len()>20].assign(quote_author=lambda x:x.quote+"|author|"+x.author).assign(tags=lambda x:x.tags.str.split(' ')).explode('tags')\
    .groupby('tags').agg(
    list_quote_same_tag=('quote_author',list),
    num_quote=('quote_author',len)
).loc[lambda x:x.num_quote>=90].sort_values('num_quote',ascending=False)

In [7]:

def extract_quote_author_list(list_author_quote, sep="|author|"):
    # print(list_author_quote)
    try:
        return [
            {"item": x[0], "owner": x[1]}
            for x in list(
                map(
                    lambda y: y.split(sep),
                    list(filter(lambda z: isinstance(z, str), list_author_quote)),
                )
            )
        ]
    except:
        print(list_author_quote)

In [8]:
# work_dir='/data/DSTeam/locnt2/personal/quotes-tagging'

In [9]:
for dt in pd.date_range(date(2023,5,27),date(2023,10,30)):
    row=df_filterd.iloc[random.randint(0,len(df_filterd)-1)]
    extracted_data=extract_quote_author_list(random.sample(row.list_quote_same_tag,k=90))
    year_month=dt.strftime('%Y%m')
    current_dt_str=dt.strftime('%Y%m%d')
    year_month_directory='../data/features/trending/{}'.format(year_month)
    if not os.path.exists(year_month_directory):
        os.makedirs(year_month_directory)

    with open(
                "../data/features/trending/{}/{}.json".format(year_month,current_dt_str), "w"
            ) as outfile:
        json.dump(
            {
                'contents':extracted_data,
                'common_tag':row.name,
                'updatedAt':datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            },
            outfile
        )